In [ ]:
!pip install torchaudio==0.13.0 torchvision==0.14.0 torch==1.13.0

In [ ]:
!pip install openmim==0.3.2
!mim install mmcv-full==1.7.0
!pip install -e .

In [ ]:
!pip install -r requirements.txt 

In [ ]:
!apt install libgl1-mesa-glx --yes

In [ ]:
!pip install mmdet==2.27.0

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from mmdet.models import build_detector
from mmdet.apis import init_detector, inference_detector
from mmcv import Config

In [ ]:
model_path = "best_bbox_h_mAP_epoch_33.pth" 
config_path = "src/deformable_detr/deformable_detr_r50_16x2_50e_coco.py"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = init_detector(config_path, model_path, device=device)

In [ ]:
frame = cv2.imread("20241203_170425.jpg")  # test frame
results = inference_detector(model, frame)
print(f"Inference results: {results}")

In [ ]:
import cv2
from tqdm import tqdm
from time import sleep

# realtime webcam
cap = cv2.VideoCapture(0)

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 30.0
print(fps)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
saved_video = cv2.VideoWriter("saved_video.mp4", fourcc, fps, (640, 480))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_resized = cv2.resize(frame_rgb, (640, 480))

    results = inference_detector(model, frame_resized)

    #print(f"Results length: {len(results)}")
    for idx, result in enumerate(results):
        for bbox in result:
            #print("Bounding box:", bbox)
            x_min, y_min, x_max, y_max, score = bbox
            if score > 0.95:  # Confidence threshold
                label = "Gun Detected"
                #print(label)
                cv2.rectangle(frame_resized, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
                cv2.putText(frame_resized, label, (int(x_min), int(y_min)-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    frame_resized = frame_resized[:, :, ::-1]
    cv2.imshow("Real-Time Detection", frame_display)
    saved_video.write(frame_display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
saved_video.release()
cv2.destroyAllWindows()

print("done")